In [1]:
from neo4j import GraphDatabase
import pandas as pd

# Connect to Neo4j
uri = "bolt://localhost:7687"  # or your remote URI
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678")) # edit password

# Define the query
# change candidate_location to your actual node label
query = """
MATCH (c:CandidateLocation)
RETURN c.lat AS Latitude, c.lon AS Longitude, c.score AS Score, c.distance_to_nearest AS DistanceToNearest
"""

# Run the query and convert results to DataFrame
with driver.session() as session:
    result = session.run(query)
    df = pd.DataFrame([r.data() for r in result])




In [2]:
# drop nan in score
df = df.dropna(subset=['Score'])
display(df["Score"].describe())
df


count    41818.000000
mean      1131.616585
std       1707.592535
min          5.645039
25%        231.383849
50%        616.253488
75%       1377.089022
max      26032.128495
Name: Score, dtype: float64

,Latitude,Longitude,Score,DistanceToNearest
0,52.017583,4.379456,474.007437,125.701512
1,52.019040,4.389782,405.307802,102.801634
2,52.025788,4.379800,2405.377351,769.491484
3,52.012111,4.380969,164.045586,22.380895
4,52.023456,4.383688,3834.628809,1245.908636
...,...,...,...,...
41813,52.002989,4.332157,112.298064,26.370222
41814,52.092089,4.401344,284.137532,65.061198
41815,52.003306,4.331589,220.862728,62.558444
41816,52.092349,4.401918,296.319810,69.121958


In [17]:
from keplergl import KeplerGl

# Create a Kepler map and add data
map_candidate_locations = KeplerGl()
map_candidate_locations.add_data(data=df, name="Candidate Locations")

# Optional: Define a custom config using score for color scaling
config = {
    "version": "v1",
    "config": {
        "visState": {
            "layers": [
                {
                    "id": "candidate-layer",
                    "type": "point",
                    "config": {
                        "dataId": "Candidate Locations",
                        "label": "EV Candidates",
                        "columns": {
                            "lat": "Latitude",
                            "lng": "Longitude",
                            "altitude": None
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 20,
                            "opacity": 0.8,
                            "filled": True,
                            "colorRange": {
                                "name": "Custom Red-Green",
                                "type": "sequential",
                                "category": "Custom",
                                "colors": [
                                    "#ff0000", "#ff4000", "#ff8000", "#ffbf00", "#ffff00",
                                    "#bfff00", "#80ff00", "#40ff00", "#00ff00"
                                ]
                            }
                        },
                        "textLabel": [
                            {
                                "field": {"name": "Name", "type": "string"},
                                "color": [255, 255, 255],
                                "size": 12,
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    },
                    "visualChannels": {
                        "colorField": {"name": "Score", "type": "real"},
                        "colorScale": "linear"
                    }
                }
            ]
        },
                    'mapStyle': {'styleType': 'voyager',
                        'topLayerGroups': {},
                        'visibleLayerGroups': {'label': True,
                            'road': True,
                            'border': False,
                            'building': True,
                            'water': True,
                            'land': True,
                            '3d building': False},
                    }
    }
}

map_candidate_locations.config = config
map_candidate_locations


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'id': 'candidate-layer', 'type': 'point'…

In [14]:
# get all the municipalities
query_municipalities = """
MATCH (m:Municipality)
RETURN m
"""

with driver.session() as session:
    result_municipalities = session.run(query_municipalities)
    municipalities = [r['m'] for r in result_municipalities]
# Create a DataFrame for municipalities
municipalities_names = [m['name'] for m in municipalities]

# create a map for each municipality
municipality_maps = {}
for municipality in municipalities_names:
    query_municipality = f"""
    MATCH (c:CandidateLocation)-[:IS_LOCATED_IN]->(m:Municipality {{name: '{municipality}'}})
    RETURN c.lat AS Latitude, c.lon AS Longitude, c.score AS Score, c.distance_to_nearest AS DistanceToNearest
    """
    
    with driver.session() as session:
        result_municipality = session.run(query_municipality)
        df_municipality = pd.DataFrame([r.data() for r in result_municipality])
        df_municipality = df_municipality.dropna(subset=['Score'])
        
        # Create a Kepler map for the municipality
        map_municipality = KeplerGl()
        map_municipality.add_data(data=df_municipality, name=f"{municipality} Candidate Locations")
        
        # Set the same config as before
        map_municipality.config = config
        
        # save the map to an HTML file
        map_municipality.save_to_html(file_name=f"maps/{municipality}_candidate_locations.html")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Hoeksche Waard_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Dordrecht_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Den Haag_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Lansingerland_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Nieuwkoop_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Katwijk_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Voorschoten_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Leidschendam-Voorburg_candidate_locations.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to maps/Leiden_candidate_

In [25]:
# get the top n candidate locations
top_n = 1000  # Change this to the number of top candidates you want
query_top_candidates = f"""
MATCH (c:CandidateLocation) - [] -> (m:Municipality)
RETURN c.lat AS Latitude, c.lon AS Longitude, c.score AS Score, c.distance_to_nearest AS DistanceToNearest, m.name AS Municipality
ORDER BY c.score DESC
LIMIT {top_n}
"""
with driver.session() as session:
    result_top_candidates = session.run(query_top_candidates)
    df_top_candidates = pd.DataFrame([r.data() for r in result_top_candidates])
    df_top_candidates = df_top_candidates.dropna(subset=['Score'])

# Create a Kepler map for top candidates
map_top_candidates = KeplerGl()
map_top_candidates.add_data(data=df_top_candidates, name="Top Candidate Locations")
# Set the same config as before
map_top_candidates.config = config  

map_top_candidates

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'id': 'candidate-layer', 'type': 'point'…

In [26]:
# table of municipalities with number of candidates in top n
municipality_counts = df_top_candidates.groupby('Municipality').size().reset_index(name='Count')
municipality_counts = municipality_counts.sort_values(by='Count', ascending=False)
# Display the table
display(municipality_counts)

,Municipality,Count
10,Hoeksche Waard,122
11,Kaag en Braassem,116
25,Rotterdam,101
29,Voorne aan Zee,89
22,Noordwijk,86
14,Krimpenerwaard,66
5,Goeree-Overflakkee,48
19,Molenlanden,44
20,Nieuwkoop,34
2,Alphen aan den Rijn,33
